In [7]:
import pandas as pd
import numpy as np
import re 

In [8]:
#raw = pd.read_csv("data\\raw.csv")
cleaned = pd.read_csv("data\gpt-3.5-expanded.en.csv")

In [9]:
cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191094 entries, 0 to 191093
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   puddle_id        191094 non-null  int64 
 1   example_id       191094 non-null  object
 2   spoken_language  191094 non-null  object
 3   country_code     188774 non-null  object
 4   sign_language    190619 non-null  object
 5   sign_writing     191094 non-null  object
 6   annotated_texts  191092 non-null  object
dtypes: int64(1), object(6)
memory usage: 10.2+ MB


In [11]:
cleaned = cleaned.drop(columns=["puddle_id","example_id","spoken_language","country_code","sign_language"])
cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191094 entries, 0 to 191093
Data columns (total 2 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   sign_writing     191094 non-null  object
 1   annotated_texts  191092 non-null  object
dtypes: object(2)
memory usage: 2.9+ MB


In [13]:
def parse_column(text):
    
    matches = re.findall(r'M|S\S{5}', text)
    
    # Elimina gli elementi duplicati 
    unique_matches = list(set(matches))
    
    # Ordina la lista mettendo "M" come primo carattere
    sorted_matches = sorted(unique_matches, key=lambda x: ('M' not in x, x))
    
    # Concatena gli elementi della lista in una stringa
    concatenated_string = ' '.join(sorted_matches)
    
    return concatenated_string


In [14]:
cleaned["sign_writing"] = cleaned["sign_writing"].apply(parse_column)

In [17]:
#Rinomino le colonne secondo lo standard

cleaned = cleaned.rename(columns={"annotated_texts": "input_text","sign_writing": "target_text"})
cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191094 entries, 0 to 191093
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   target_text  191094 non-null  object
 1   input_text   191092 non-null  object
dtypes: object(2)
memory usage: 2.9+ MB


In [18]:
#Rimuovo eventuali missing

cleaned = cleaned.dropna()
cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 191092 entries, 0 to 191093
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   target_text  191092 non-null  object
 1   input_text   191092 non-null  object
dtypes: object(2)
memory usage: 4.4+ MB


In [20]:
#Preparo il dataframe per lo standard richiesto dal modello google-t5

cleaned['id'] = cleaned.reset_index().index
cleaned['id'] = cleaned['id'].astype(str)
cleaned = cleaned[['id', 'input_text', 'target_text']]

In [21]:
cleaned['translation'] = cleaned.apply(lambda row: {'en': row['input_text'], 'fsw': row['target_text']}, axis=1)

In [22]:
cleaned = cleaned.drop(columns=["input_text","target_text"])

In [23]:
cleaned

,id,translation
0,0,"{'en': 'I'm going to cry', 'fsw': 'MS20320S21d..."
1,1,"{'en': 'qua', 'fsw': 'MS10b43S26504S2f710S36d00'}"
2,2,"{'en': 'turn off', 'fsw': 'MS1ee40S1ee48S22100..."
3,3,"{'en': 'street', 'fsw': 'MS15d40S15d48S26600S2..."
4,4,"{'en': 'bathroom', 'fsw': 'MS20313S25505S36e03'}"
...,...,...
191089,191087,"{'en': 'today', 'fsw': 'MS15a31S15a39'}"
191090,191088,"{'en': 'yesterday', 'fsw': 'MS15a00S26604S2f70..."
191091,191089,"{'en': 'tomorrow', 'fsw': 'MS10020S10040S20500..."
191092,191090,"{'en': 'morning', 'fsw': 'MS15a20S15a22S15a26S..."


In [24]:
#Splitting del dataset

train_df = cleaned.sample(frac=0.8, random_state=42)
remaining_df = cleaned.drop(train_df.index)
valid_df = remaining_df.sample(frac=0.5, random_state=42)
test_df = remaining_df.drop(valid_df.index)

In [25]:
from datasets import Dataset, DatasetDict
train_df.reset_index(drop=True, inplace=True)
valid_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

train_data = Dataset.from_pandas(train_df)
validation_data = Dataset.from_pandas(valid_df)
test_data = Dataset.from_pandas(test_df)

In [26]:
test_data

Dataset({
    features: ['id', 'translation'],
    num_rows: 19109
})

In [27]:
train_data

Dataset({
    features: ['id', 'translation'],
    num_rows: 152874
})

In [28]:
validation_data

Dataset({
    features: ['id', 'translation'],
    num_rows: 19109
})

In [29]:
import torch
 
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer
)

In [30]:
#Loading del modello

MODEL = 't5-base'
tokenizer = T5Tokenizer.from_pretrained(MODEL)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [31]:
source_lang = "en"
target_lang = "fsw"
prefix = "translate English to FSW: "

In [32]:
#Creo il dataset dictionary

big = DatasetDict({
    "train": train_data,
    "valid": validation_data,
    "test": test_data
})

In [32]:
big

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 152874
    })
    valid: Dataset({
        features: ['id', 'translation'],
        num_rows: 19109
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 19109
    })
})

In [33]:
print(big["train"][0])

{'id': '164198', 'translation': {'en': 'Emotion᛫Emotional', 'fsw': 'MS15021S1502fS20351S26525S30102'}}


In [34]:
#Preprocess function di huggingface

def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [35]:
#Tokenizzazione del dataset

tokenized_train = big.map(
    preprocess_function,
    batched=True
)

Map:   0%|          | 0/152874 [00:00<?, ? examples/s]

Map: 100%|██████████| 19109/19109 [00:01<00:00, 14919.41 examples/s]


In [36]:
tokenized_train["train"]

Dataset({
    features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 152874
})

In [37]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=MODEL)

In [38]:
import evaluate

metric = evaluate.load("sacrebleu")

In [39]:
metric

EvaluationModule(name: "sacrebleu", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions (`list` of `str`): list of translations to score. Each translation should be tokenized into a list of tokens.
    references (`list` of `list` of `str`): A list of lists of references. The contents of the first sub-list are the references for the first prediction, the contents of the second sub-list are for the second prediction, etc. Note that there must be the same number of references for each prediction (i.e. all sub-lists must be of the same length).
    smooth_method (`str`): The smoothing method to use, defaults to `'e

In [41]:
#Funzioni di huggingface

import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [42]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL)

c:\Users\Micaela Lisa Masano\Desktop\NLP\nlp\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Micaela Lisa Masano\.cache\huggingface\hub\models--t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [43]:
#imposto training su gpu se possibile

import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [44]:
device

device(type='cuda')

In [45]:
training_args = Seq2SeqTrainingArguments(
    output_dir="t5_en_to_fsw_model_trained",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True
)

In [46]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train["train"],
    eval_dataset=tokenized_train["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

c:\Users\Micaela Lisa Masano\Desktop\NLP\nlp\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [47]:
trainer.train()

  0%|          | 0/19110 [00:00<?, ?it/s]

  3%|▎         | 500/19110 [02:09<1:14:13,  4.18it/s]

{'loss': 2.6676, 'grad_norm': 1.6579103469848633, 'learning_rate': 1.947880690737834e-05, 'epoch': 0.05}


  5%|▌         | 1000/19110 [05:08<2:23:42,  2.10it/s]

{'loss': 2.0503, 'grad_norm': 1.9872941970825195, 'learning_rate': 1.8955520669806387e-05, 'epoch': 0.1}


...

 99%|█████████▉| 19000/19110 [1:51:45<01:21,  1.36it/s]

{'loss': 1.5366, 'grad_norm': 1.373734474182129, 'learning_rate': 1.2244897959183673e-07, 'epoch': 1.99}


100%|██████████| 19110/19110 [1:52:20<00:00,  3.71it/s]c:\Users\Micaela Lisa Masano\Desktop\NLP\nlp\Lib\site-packages\transformers\generation\utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
                                                       
100%|██████████| 19110/19110 [2:11:35<00:00,  2.42it/s]

{'eval_loss': 1.4305686950683594, 'eval_bleu': 0.0, 'eval_gen_len': 18.3857, 'eval_runtime': 1154.1486, 'eval_samples_per_second': 16.557, 'eval_steps_per_second': 1.035, 'epoch': 2.0}
{'train_runtime': 7895.115, 'train_samples_per_second': 38.726, 'train_steps_per_second': 2.42, 'train_loss': 1.654687969501202, 'epoch': 2.0}


TrainOutput(global_step=19110, training_loss=1.654687969501202, metrics={'train_runtime': 7895.115, 'train_samples_per_second': 38.726, 'train_steps_per_second': 2.42, 'train_loss': 1.654687969501202, 'epoch': 2.0})

In [4]:
#Load del modello dopo fine tuning

model_name = "t5_model_trained\checkpoint-95500"
model_base = "google-t5/t5-base"
model_tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [64]:
#Inferenza sulla singola frase

text_to_translate = "translate English to FSW: Hello word"
inputs = model_tokenizer.encode(text_to_translate, return_tensors="pt")
outputs = model.generate(inputs, max_length=128, num_beams=4, early_stopping=True)
translated_text = model_tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Traduzione:", translated_text)

Traduzione: MS14c20S14c28S26500S26510S2fb04S33e00


In [47]:
big["train"]["translation"][0]

{'en': 'Emotion᛫Emotional', 'fsw': 'MS15021S1502fS20351S26525S30102'}

In [58]:
df = pd.DataFrame(test_df)

In [59]:
prefix = "Translate English to FSW: "
df["to_translate"] = df["translation"].apply(lambda x: prefix + x["en"])

In [60]:
df["ground_t"] = df["translation"].apply(lambda x: x["fsw"])

In [68]:
#Conversione delle frasi del test set

translated_phrases = []
for sentence in df["to_translate"]:
    inputs = model_tokenizer.encode(sentence, return_tensors="pt")
    outputs = model.generate(inputs, max_length=128, num_beams=4, early_stopping=True)
    translated_text = model_tokenizer.decode(outputs[0], skip_special_tokens=True)
    translated_phrases.append(translated_text)

KeyboardInterrupt: 

In [69]:
sentence = "Translate English to FSW: I love pizza"

inputs = model_tokenizer.encode(sentence, return_tensors="pt")
outputs = model.generate(inputs, max_length=128, num_beams=4, early_stopping=True)
translated_text = model_tokenizer.decode(outputs[0], skip_special_tokens=True)
translated_phrases.append(translated_text)

,id,translation,to_translate,ground_t
0,15,"{'en': 'nero', 'fsw': 'MS15d01S2e605S2ff00'}",Translate English to FSW: nero,MS15d01S2e605S2ff00
1,26,"{'en': 'pipi', 'fsw': 'MS11520S23602S33b00'}",Translate English to FSW: pipi,MS11520S23602S33b00
2,43,"{'en': 'test᛫try to do something, attempt to d...",Translate English to FSW: test᛫try to do somet...,MS10e31S15d39S21100S26a05S30a00
3,45,"{'en': 'public', 'fsw': 'MS14c32S14c3aS2df08S2...",Translate English to FSW: public,MS14c32S14c3aS2df08S2df10S2fb04
4,46,"{'en': 'schedule᛫timetable᛫time', 'fsw': 'MS14...",Translate English to FSW: schedule᛫timetable᛫time,MS14c20S14c28S22f24
...,...,...,...,...
19104,191049,"{'en': 'sword᛫to find᛫search', 'fsw': 'MS10e50...",Translate English to FSW: sword᛫to find᛫search,MS10e50S21410S2ee02S31a00S32104
19105,191058,"{'en': 'Where', 'fsw': 'MS1f550S1f558S2a208S2a...",Translate English to FSW: Where,MS1f550S1f558S2a208S2a210
19106,191067,"{'en': 'I᛫me᛫myself', 'fsw': 'MS10044S20500'}",Translate English to FSW: I᛫me᛫myself,MS10044S20500
19107,191072,"{'en': 'Applaud᛫Cheer᛫Hooray', 'fsw': 'MS15210...",Translate English to FSW: Applaud᛫Cheer᛫Hooray,MS15210S15218S2e234S34000


In [72]:
translated_phrases[0]

'MS14c02S14c0aS21100S22a04S22a14S2fb04S2ff00'

In [84]:
#Calcolo precision recall e F1

from sklearn.metrics import precision_recall_fscore_support

def calculate_metrics(generated_sentences, reference_labels):

    precision, recall, f1_score, _ = precision_recall_fscore_support(reference_labels, generated_sentences, average= "weighted")


    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-score:", f1_score)


calculate_metrics(translated_phrases, df["ground_t"][:679,])

Precision: 0.0034364261168384875
Recall: 0.005891016200294551
F1-score: 0.004172803141875307


c:\Users\Ale\Desktop\Università\NLP\nlp_cuda\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Ale\Desktop\Università\NLP\nlp_cuda\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
